In [1]:
# Define a Neural Network
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # an affine operation
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
# Output learnable parameters of a model
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [3]:
# nn.Conv2d takes a 4D Tensor of nSamples x nChannels x Height x Width
# For a single sample, use input.unsqueeze(0) to add a fake batch dimension

input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0983, -0.1187,  0.0110,  0.0761,  0.0931, -0.0487,  0.1999, -0.0101,
         -0.0978,  0.0867]], grad_fn=<AddmmBackward>)


In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [5]:
# Compute loss
output = net(input)
print(output)

target = torch.randn(10)
print(target)

target = target.view(1, -1)
print(target)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor([[-0.0983, -0.1187,  0.0110,  0.0761,  0.0931, -0.0487,  0.1999, -0.0101,
         -0.0978,  0.0867]], grad_fn=<AddmmBackward>)
tensor([ 0.6649, -1.2109, -2.8296,  1.2648, -0.3321,  0.1779, -0.6070,  0.5593,
        -0.3678, -0.2054])
tensor([[ 0.6649, -1.2109, -2.8296,  1.2648, -0.3321,  0.1779, -0.6070,  0.5593,
         -0.3678, -0.2054]])
tensor(1.2623, grad_fn=<MseLossBackward>)


In [6]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [7]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0252, -0.0109,  0.0108,  0.0112,  0.0357,  0.0038])


In [8]:
# Update the weights
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [9]:
import torch.optim as optim

# Create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)

loss = criterion(output, target)
loss.backward()
optimizer.step() # does the update